In [ ]:
# Imports & paths
import os
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from astropy.table import Table

cat_dir     = r"pass to/catalogs folder"
fits_path   = os.path.join(cat_dir, "your_file_name.fits")

In [ ]:
#Load core tables
with fits.open(fits_path, memmap=True) as hdul:
    T_slices = Table(hdul["SLICES"].data)       # slice metadata
    T_points = Table(hdul["POINTS"].data)       # per-slice objects (RA/Dec, etc.)
    T_over   = Table(hdul["OVERDENSITY"].data)  # per-object overdensity (id, density_excess)

print(len(T_slices), "slices,", len(T_over), "overdensity rows")

In [ ]:
# load per object overdensity table
with fits.open(fits_path, memmap=True) as hdul:
    T_over = Table(hdul["OVERDENSITY"].data)

T_over.info()
T_over[:10]   # preview first few rows

In [ ]:
# helper: get a map for a given slice index
def get_slice_map(slice_index, map_type="DENSITY", rotated=False):
    """
    slice_index : integer slice number (as in SLICES['slice'])
    map_type    : 'DENSITY', 'DENSITYCONTRAST', or 'DENSITYEXCESS'
    rotated     : False → sky frame; True → internal rotated frame
    returns     : (array, header)
    """
    with fits.open(fits_path, memmap=True) as hdul:
        # construct expected HDU name
        if rotated:
            base = {
                "DENSITY":          "DensityRotated",
                "DENSITYCONTRAST":  "DensityContrastRotated",
                "DENSITYEXCESS":    "DensityExcessRotated",
            }[map_type]
        else:
            base = map_type

        hdu_name = f"{base}_{slice_index:03d}"

        if hdu_name in hdul:
            hdu = hdul[hdu_name]
            return np.asarray(hdu.data), hdu.header

        # fallback: search by header keywords if names ever change
        for hdu in hdul:
            hdr = getattr(hdu, "header", None)
            if hdr is None:
                continue
            if hdr.get("SLICE") == slice_index and hdr.get("IMTYPE") == map_type:
                return np.asarray(hdu.data), hdr

    raise KeyError(f"Map not found for slice={slice_index}, map_type={map_type}, rotated={rotated}")


# example: get slice 10 density map in sky frame
arr, hdr = get_slice_map(slice_index=10, map_type="DENSITY", rotated=False)

print("shape:", arr.shape)
print("slice:", hdr.get("SLICE"), "z:", hdr.get("Z"), "bg_dens:", hdr.get("BG_DENS"))
print("min/max:", float(np.nanmin(arr)), float(np.nanmax(arr)))


In [ ]:
# load POINTS once (for RA/Dec extent)
with fits.open(fits_path, memmap=True) as hdul:
    T_points = Table(hdul["POINTS"].data)


# Plotting Overdensity Map for a Given Slice
def plot_slice_density(slice_index, map_type="DENSITYEXCESS"):
    """
    slice_index : integer slice index (as in SLICES['slice'])
    map_type    : 'DENSITY', 'DENSITYCONTRAST', or 'DENSITYEXCESS'
    """
    # 2D map from the FITS (already in RA/Dec frame)
    arr, hdr = get_slice_map(slice_index, map_type=map_type, rotated=False)

    # RA/Dec extent from POINTS table for this slice
    mask = (T_points["slice"] == slice_index)
    ra   = T_points["RA"][mask]
    dec  = T_points["Dec"][mask]

    x_min, x_max = float(ra.min()),  float(ra.max())
    y_min, y_max = float(dec.min()), float(dec.max())

    # log color scaling
    pos_vals = arr[arr > 0]
    vmin = float(pos_vals.min()) if pos_vals.size else np.finfo(float).eps
    vmax = float(arr.max())

    fig, ax = plt.subplots(figsize=(7, 6))
    im = ax.imshow(
        arr,
        extent=(x_min, x_max, y_min, y_max),
        origin="lower",
        aspect="auto",
        cmap="jet",
        norm=LogNorm(vmin=vmin, vmax=vmax),
    )

    ax.invert_xaxis()  # RA increasing to the left
    z_slice = hdr.get("Z")
    ax.set_xlabel("RA [deg]", fontsize=14)
    ax.set_ylabel("Dec [deg]", fontsize=14)
    ax.set_title(f"slice {slice_index},   z = {z_slice:.2f}", fontsize=15)

    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label(r"1 + $\delta$", fontsize=14)
    allowed_ticks = [0.1, 1, 10]
    ticks = [t for t in allowed_ticks if vmin <= t <= vmax]
    if ticks:
        cbar.set_ticks(ticks)
        cbar.set_ticklabels([str(t) for t in ticks])

    plt.tight_layout()
    plt.show()

# example: plot slice 10, 1+delta map
plot_slice_density(10)
